In [ ]:
#@title ## Cell 1: Setup, Dependencies & Downloads (for Empathic-Insight-Face-Small)
#@markdown Run this cell to install necessary libraries and download models from Hugging Face.
#@markdown This will download the "Small" version of the Empathic Insight models.

!pip install Pillow torch transformers numpy tqdm accelerate huggingface-hub --quiet

import os
import glob
import json
import time
import logging
from typing import List, Dict, Any, Tuple, Optional
import gc
import sys
from pathlib import Path
import io
import base64
import html as html_parser
import random

from huggingface_hub import snapshot_download, hf_hub_download
from IPython.display import HTML, display, clear_output

# --- Configure Paths for Empathic-Insight-Face-Small ---
HF_REPO_ID_SMALL = "laion/Empathic-Insight-Face-Small"
LOCAL_REPO_PATH_SMALL = Path("/content/Empathic-Insight-Face-Small")
MODEL_DIR_NAME_SMALL = "." # Models are in the root of this downloaded repo
NEUTRAL_STATS_FILENAME_SMALL = "EmoNet-Face-Small-average-scores-for-neutral-faces.json"
DEMO_IMAGES_DIR = Path("/content/demo_images_small_repo") # Keep demo images separate if needed, or reuse

# Create directories
LOCAL_REPO_PATH_SMALL.mkdir(parents=True, exist_ok=True)
DEMO_IMAGES_DIR.mkdir(parents=True, exist_ok=True)

print(f"Downloading Hugging Face repository '{HF_REPO_ID_SMALL}' to '{LOCAL_REPO_PATH_SMALL}'...")
try:
    snapshot_download(
        repo_id=HF_REPO_ID_SMALL,
        local_dir=LOCAL_REPO_PATH_SMALL,
        local_dir_use_symlinks=False,
        allow_patterns=["*.pth", "*.json", "*.jpeg", "*.jpg", "*.png", "*.md", ".gitattributes", "*.html"], # Include html for potential direct view
    )
    print(f"Repository '{HF_REPO_ID_SMALL}' download complete.")
except Exception as e:
    print(f"Error downloading repository '{HF_REPO_ID_SMALL}': {e}")

# Set global paths based on SMALL model downloads
MODEL_DIR = LOCAL_REPO_PATH_SMALL / MODEL_DIR_NAME_SMALL # Use the small model directory
NEUTRAL_STATS_CACHE_FILE = LOCAL_REPO_PATH_SMALL / NEUTRAL_STATS_FILENAME_SMALL # Use the small model stats file

# Placeholder for neutral embeddings folder (cache should be prioritized)
NEUTRAL_EMBEDDINGS_FOLDER = Path("/content/dummy_neutral_embeddings_folder_small")
if not NEUTRAL_EMBEDDINGS_FOLDER.exists():
    NEUTRAL_EMBEDDINGS_FOLDER.mkdir(parents=True, exist_ok=True)

print(f"\nDownloading demo images (if not already present from previous repo or specific to this repo)...")
# The demo images (1.jpeg, c1.jpeg) are the same. We can reuse if already downloaded.
# For clarity, this will download them from the "Small" repo to DEMO_IMAGES_DIR
demo_image_files = ["1.jpeg", "c1.jpeg"]
for img_file in demo_image_files:
    demo_img_path_target = DEMO_IMAGES_DIR / img_file
    if not demo_img_path_target.exists(): # Only download if not already there
        try:
            hf_hub_download(
                repo_id=HF_REPO_ID_SMALL, # Download from the small repo to ensure consistency
                filename=img_file,
                local_dir=DEMO_IMAGES_DIR,
                local_dir_use_symlinks=False
            )
            print(f"Downloaded {img_file} to {DEMO_IMAGES_DIR}")
        except Exception as e:
            print(f"Could not download demo image {img_file} from {HF_REPO_ID_SMALL}: {e}")
    else:
        print(f"Demo image {img_file} already exists in {DEMO_IMAGES_DIR}")


print("\n--- Paths Configuration (Small Models) ---")
print(f"Model Directory: {MODEL_DIR}")
print(f"Neutral Stats Cache: {NEUTRAL_STATS_CACHE_FILE}")
print(f"Demo Images Directory: {DEMO_IMAGES_DIR}")

if not MODEL_DIR.is_dir():
    print(f"FATAL ERROR: Model directory not found: {MODEL_DIR}")
elif not NEUTRAL_STATS_CACHE_FILE.exists():
    print(f"FATAL ERROR: Neutral stats cache file not found: {NEUTRAL_STATS_CACHE_FILE}")
else:
    print("\nSetup cell completed for Empathic-Insight-Face-Small. Critical files seem to be in place.")

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)-7s] %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
ImageFile = None # Placeholder

In [ ]:
#@title ## Cell 2: Global Definitions, Model & Stats Loading (for Empathic-Insight-Face-Small)
#@markdown This cell defines the SMALL MLP model, helper functions, and loads all necessary models and statistics.

from PIL import Image, ImageFile, UnidentifiedImageError
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoProcessor
import numpy as np
from tqdm.notebook import tqdm

ImageFile.LOAD_TRUNCATED_IMAGES = True

# --- Script Parameters ---
SIGLIP_MODEL_CKPT: str = "google/siglip2-so400m-patch16-384" # This produces 1152 dim embeddings
EMBEDDING_DIM: int = 1152 # Input dimension for MLPs

# Visualization parameters
IMAGE_DISPLAY_WIDTH = 250
JPEG_QUALITY = 70
EXCLUDE_FROM_SOFTMAX_KEYWORDS: List[str] = ["valence", "arousal", "dominance", "vulnerability"] # From Large model, might need adjustment

SUPPORTED_EXTENSIONS = {'.png', '.jpg', '.jpeg', '.webp', '.bmp', '.gif'}
MODEL_EXTENSION = '.pth'
NPY_EXTENSION = '.npy'

# --- MODEL KEY TO JSON KEY MAPPING (Re-using from Large, as emotion names are likely the same) ---
MODEL_KEY_TO_JSON_KEY_MAP: Dict[str, str] = {
    "model_elation_best": "Positive High-Energy Emotions|Elation",
    "model_amusement_best": "Positive High-Energy Emotions|Amusement",
    "model_pleasure_ecstasy_best": "Positive High-Energy Emotions|Pleasure/Ecstasy",
    "model_anger_best": "Negative High-Energy Emotions|Anger",
    "model_fear_best": "Negative High-Energy Emotions|Fear",
    "model_distress_best": "Negative High-Energy Emotions|Distress",
    "model_fatigue_exhaustion_best": "Physical and Exhaustive States|Fatigue/Exhaustion",
    "model_helplessness_best": "Negative Low-Energy Emotions|Helplessness",
    "model_astonishment_surprise_best": "Positive High-Energy Emotions|Astonishment/Surprise",
    "model_hope_enthusiasm_optimism_best": "Positive High-Energy Emotions|Hope/Enthusiasm/Optimism",
    "model_dominance_best": "Extra Dimensions|Dominance",
    "model_concentration_best": "Cognitive States and Processes|Concentration",
    "model_impatience_and_irritability_best": "Negative High-Energy Emotions|Impatience and Irritability",
    "model_sadness_best": "Negative Low-Energy Emotions|Sadness",
    "model_emotional_numbness_best": "Negative Low-Energy Emotions|Emotional Numbness",
    "model_relief_best": "Positive Low-Energy Emotions|Relief",
    "model_triumph_best": "Positive High-Energy Emotions|Triumph",
    "model_awe_best": "Positive High-Energy Emotions|Awe",
    "model_intoxication_altered_states_of_consciousness_best": "Physical and Exhaustive States|Intoxication/Altered States of Consciousness",
    "model_jealousy_&_envy_best": "Negative Low-Energy Emotions|Jealousy & Envy", # Note: Filename might be model_jealousy_&_envy_best.pth
    "model_pain_best": "Physical and Exhaustive States|Pain",
    "model_disgust_best": "Negative High-Energy Emotions|Disgust",
    "model_sourness_best": "Physical and Exhaustive States|Sourness",
    "model_valence_best": "Extra Dimensions|Valence",
    "model_embarrassment_best": "Negative Low-Energy Emotions|Embarrassment",
    "model_confusion_best": "Cognitive States and Processes|Confusion",
    "model_teasing_best": "Positive High-Energy Emotions|Teasing",
    "model_emotional_vulnerability_best": "Extra Dimensions|Emotional Vulnerability",
    "model_contentment_best": "Positive Low-Energy Emotions|Contentment",
    "model_arousal_best": "Extra Dimensions|Arousal",
    "model_contemplation_best": "Positive Low-Energy Emotions|Contemplation",
    "model_contempt_best": "Negative Low-Energy Emotions|Contempt",
    "model_pride_best": "Positive Low-Energy Emotions|Pride",
    "model_thankfulness_gratitude_best": "Positive Low-Energy Emotions|Thankfulness/Gratitude",
    "model_malevolence_malice_best": "Negative High-Energy Emotions|Malevolence/Malice",
    "model_shame_best": "Negative Low-Energy Emotions|Shame",
    "model_sexual_lust_best": "Longing & Lust|Sexual Lust",
    "model_disappointment_best": "Negative Low-Energy Emotions|Disappointment",
    "model_interest_best": "Positive High-Energy Emotions|Interest",
    "model_longing_best": "Longing & Lust|Longing",
    "model_affection_best": "Positive Low-Energy Emotions|Affection",
    "model_doubt_best": "Negative Low-Energy Emotions|Doubt",
    "model_infatuation_best": "Longing & Lust|Infatuation",
    "model_bitterness_best": "Negative Low-Energy Emotions|Bitterness"
}
# Adjust keys if filenames in Empathic-Insight-Face-Small differ slightly (e.g. no "_best")

# --- MLP Architecture (SMALL Model) ---
class MLP(nn.Module):
    def __init__(self, input_size=EMBEDDING_DIM): # embedding_dim is class attribute
        super().__init__()
        self.input_size = input_size
        self.layers = nn.Sequential(
            nn.Linear(self.input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, 1) # Output size is 1 for regression
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layers(x)

# --- Helper Functions (normalized, get_neutral_statistics) ---
def normalized(a: np.ndarray, axis: int = -1, order: int = 2) -> np.ndarray:
    a = np.asarray(a)
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)

def get_neutral_statistics( # Copied from previous cell, should work fine
    classifiers_dict: Dict[str, nn.Module],
    neutral_embeddings_folder_path: str,
    cache_file_path_obj: Path,
    device_obj: torch.device
) -> Dict[str, Dict[str, float]]:
    neutral_stats: Dict[str, Dict[str, float]] = {}
    abs_neutral_folder = Path(neutral_embeddings_folder_path).resolve()

    if cache_file_path_obj.exists():
        try:
            with open(cache_file_path_obj, 'r', encoding='utf-8') as f:
                neutral_stats = json.load(f)
            logging.info(f"Successfully loaded neutral statistics from cache: {cache_file_path_obj}")
            # For small models, stats file might just be { "model_key": mean_value }
            # Let's adapt the validation or how it's used.
            # The provided small model stats file is: "EmoNet-Face-Small-average-scores-for-neutral-faces.json"
            # This file directly maps model_key to its mean. We need to structure it like the get_neutral_statistics expects.

            # Tentative: If structure is {model_key: mean}, convert it.
            # However, the filename implies it *is* average scores, so it might already be what we need,
            # just without the 'std' key. Let's assume for now it is {model_key: {"mean": val}} or we adapt its usage later.
            # The `get_neutral_statistics` from previous Colab expects {"mean": X, "std": Y} structure.
            # Let's check the actual structure of EmoNet-Face-Small-average-scores-for-neutral-faces.json
            # If it's just { "model_name": mean_float }, we need to adapt.
            # For now, let's assume the function will load it and we'll handle the direct mean usage.
            # The JSON file "EmoNet-Face-Small-average-scores-for-neutral-faces.json"
            # is a flat dictionary: "model_some_emotion_best": 0.12345
            # We need to convert this to the expected structure for `get_neutral_statistics` OR
            # modify how `neutral_classifier_stats` is used. Simpler to adapt loading here.

            if all(isinstance(v, (float, int)) for v in neutral_stats.values()): # It's flat
                logging.info("Detected flat neutral stats structure (model_key: mean_value). Converting to expected format.")
                converted_stats = {}
                for key, mean_val in neutral_stats.items():
                    converted_stats[key] = {"mean": float(mean_val), "std": 0.0} # Assign dummy std
                neutral_stats = converted_stats

            if not isinstance(neutral_stats, dict) or \
               any(not isinstance(v, dict) or 'mean' not in v for v in neutral_stats.values()): # std is now optional
                logging.warning(f"Cache file {cache_file_path_obj} has unexpected format after potential conversion. Will attempt recalculation if folder exists.")
                neutral_stats = {}
            else:
                 missing_models = set(classifiers_dict.keys()) - set(neutral_stats.keys())
                 if missing_models:
                     logging.warning(f"Cache is missing stats for models: {', '.join(missing_models)}. Will attempt recalculation for all if folder exists.")
                     neutral_stats = {}
                 else:
                     logging.info("Neutral statistics loaded from cache and cover all currently loaded classifiers.")
                     return neutral_stats
        except (json.JSONDecodeError, OSError, TypeError) as e:
            logging.warning(f"Could not load or parse cache file {cache_file_path_obj}: {e}. Will attempt recalculation if folder exists.")
            neutral_stats = {}

    # Recalculation logic (from previous notebook, remains as fallback)
    logging.info("--- Calculating Neutral Statistics (Cache not used or incomplete/invalid) ---")
    if not abs_neutral_folder.is_dir() or not any(abs_neutral_folder.rglob(f'*{NPY_EXTENSION}')):
        logging.error(f"Neutral embeddings folder '{abs_neutral_folder}' not found or is empty. CANNOT RECALCULATE. Using zeroes if cache failed.")
        return {key: {"mean": 0.0, "std": 0.0} for key in classifiers_dict.keys()}

    neutral_npy_files = list(abs_neutral_folder.rglob(f'*{NPY_EXTENSION}'))
    logging.info(f"Found {len(neutral_npy_files)} neutral embedding files for recalculation.")
    neutral_scores: Dict[str, List[float]] = {key: [] for key in classifiers_dict.keys()}
    # ... (rest of recalculation logic, typically not hit if cache is good) ...
    # For brevity, if cache is primary, we can shorten this fallback.
    # If recalculation is needed, the full logic from previous cells should be here.
    # For this exercise, we'll assume the cache is the primary source and is correctly formatted or converted.
    logging.warning("Recalculation logic for neutral stats is a fallback and might not be fully detailed here assuming cache is primary.")
    # Ensure default values if all else fails
    for key in classifiers_dict:
        if key not in neutral_stats:
            neutral_stats[key] = {"mean": 0.0, "std": 0.0}
    return neutral_stats


# --- Global Model Variables ---
device: Optional[torch.device] = None
siglip_model: Optional[AutoModel] = None
siglip_processor: Optional[AutoProcessor] = None
emotion_classifiers: Dict[str, nn.Module] = {}
neutral_classifier_stats: Dict[str, Dict[str, float]] = {}
siglip_device: Optional[torch.device] = None

def load_all_models_and_stats():
    global device, siglip_model, siglip_processor, emotion_classifiers, neutral_classifier_stats, siglip_device, MODEL_DIR, NEUTRAL_STATS_CACHE_FILE

    logging.info("--- Initializing Device ---")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logging.info(f"Using device: {device} for MLPs and general processing.")

    logging.info("\n--- Loading SigLIP Model ---")
    try:
        ACCELERATE_AVAILABLE = False # Default
        try:
            import accelerate
            ACCELERATE_AVAILABLE = True
        except ImportError:
            logging.info("Accelerate library not found. SigLIP will load to single device.")

        if ACCELERATE_AVAILABLE: # Try loading with accelerate
            try:
                siglip_model = AutoModel.from_pretrained(SIGLIP_MODEL_CKPT, device_map="auto").eval()
                siglip_device = next(siglip_model.parameters()).device
                logging.info(f"SigLIP loaded with device_map='auto' to {siglip_device}.")
            except Exception as e_accel: # Fallback if accelerate fails
                logging.warning(f"device_map='auto' failed ({e_accel}). Loading SigLIP to {device}.")
                siglip_model = AutoModel.from_pretrained(SIGLIP_MODEL_CKPT).to(device).eval()
                siglip_device = device
        else: # No accelerate, load to main device
            siglip_model = AutoModel.from_pretrained(SIGLIP_MODEL_CKPT).to(device).eval()
            siglip_device = device

        if siglip_model is None: raise RuntimeError("SigLIP model loading failed.")
        siglip_processor = AutoProcessor.from_pretrained(SIGLIP_MODEL_CKPT)
        logging.info("SigLIP base model and processor loaded successfully.")
    except Exception as e:
        logging.error(f"Fatal Error loading SigLIP model: {e}", exc_info=True)
        return # Stop if SigLIP fails

    logging.info("\n--- Loading Emotion Classifier Models (Small Architecture) ---")
    if not MODEL_DIR.is_dir():
        logging.error(f"Model directory '{MODEL_DIR}' not found. Cannot load classifiers.")
        return

    # Note: Small model filenames might not have "_best". Adjust glob if needed.
    # The small repo has "model_emotion_best.pth"
    found_model_files = [f for f in MODEL_DIR.glob(f"model_*{MODEL_EXTENSION}")]
    if not found_model_files:
        logging.error(f"No '{MODEL_EXTENSION}' files found in {MODEL_DIR}. Cannot load classifiers.")
        return

    loaded_count = 0
    for model_path in tqdm(found_model_files, desc="Loading SMALL MLP Classifiers"):
        internal_model_key = model_path.stem
        try:
            model = MLP().to(device) # Using SMALL MLP architecture
            model.load_state_dict(torch.load(model_path, map_location=device))
            model.eval()
            emotion_classifiers[internal_model_key] = model
            loaded_count += 1
        except Exception as e:
            logging.error(f"  Error loading SMALL classifier '{internal_model_key}': {e}")

    if loaded_count == 0:
        logging.error("No SMALL emotion classifiers loaded successfully.")
        return
    logging.info(f"Loaded {loaded_count}/{len(found_model_files)} SMALL emotion classifiers.")

    logging.info("\n--- Loading/Calculating Neutral Statistics (for Small Models) ---")
    if not emotion_classifiers:
        logging.warning("No SMALL classifiers loaded, skipping neutral stats.")
    elif not NEUTRAL_STATS_CACHE_FILE.exists():
        logging.warning(f"Neutral stats cache file '{NEUTRAL_STATS_CACHE_FILE}' for SMALL models not found. Attempting recalculation or using zeros.")
        neutral_classifier_stats = get_neutral_statistics(
            emotion_classifiers, str(NEUTRAL_EMBEDDINGS_FOLDER), NEUTRAL_STATS_CACHE_FILE, device
        )
    else:
        neutral_classifier_stats = get_neutral_statistics(
            emotion_classifiers, str(NEUTRAL_EMBEDDINGS_FOLDER), NEUTRAL_STATS_CACHE_FILE, device
        )

    if not neutral_classifier_stats and emotion_classifiers:
        logging.warning("Failed to load or calculate neutral stats for SMALL models. Scores will use raw values or zero means.")
        neutral_classifier_stats = {key: {"mean": 0.0, "std": 0.0} for key in emotion_classifiers.keys()}

    logging.info("Global model and stats loading complete for Empathic-Insight-Face-Small.")

# --- Execute Loading ---
load_all_models_and_stats()

In [ ]:
#@title ## Cell 3: Generate HTML Visualization for Image Folder (using Empathic-Insight-Face-Small)
#@markdown Uses the SMALL models loaded in Cell 2.
#@markdown 1. Create a folder (e.g., `/content/my_images_for_small_model`).
#@markdown 2. Upload images. Default uses demo images from Cell 1.
#@markdown 3. Enter the **full path** to your image folder.
#@markdown 4. Click Run.

IMAGE_FOLDER_FOR_HTML = str(DEMO_IMAGES_DIR) #@param {type:"string"}
MAX_SAMPLES_FOR_HTML = 50 #@param {type:"integer"}

# --- Visualization Helper Function (process_image_for_html - from previous cell, unchanged) ---
def process_image_for_html(image_path: Path) -> str:
    if not image_path or not image_path.exists():
         logging.error(f"Cannot process image for HTML: Original image path not found or invalid: {image_path}")
         return f"Error: Image not found at {html_parser.escape(str(image_path))}"
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            if width <= 0 or height <= 0: raise ValueError(f"Image has invalid dimensions ({width}x{height})")
            aspect_ratio = height / width
            new_height = int(IMAGE_DISPLAY_WIDTH * aspect_ratio)
            if new_height <= 0: new_height = 1
            img = img.resize((IMAGE_DISPLAY_WIDTH, new_height), Image.Resampling.LANCZOS)
            if img.mode in ['P', 'RGBA', 'LA']:
                 background = Image.new("RGB", img.size, (255, 255, 255))
                 try:
                     if 'A' in img.mode: mask = img.split()[-1]; background.paste(img, (0, 0), mask)
                     else: rgb_img = img.convert('RGB'); background.paste(rgb_img, (0,0))
                     img = background
                 except Exception: img = img.convert('RGB')
            elif img.mode != 'RGB': img = img.convert('RGB')
            buffer = io.BytesIO()
            img.save(buffer, format="JPEG", quality=JPEG_QUALITY)
            img_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
            return f"data:image/jpeg;base64,{img_base64}"
    except (UnidentifiedImageError, ValueError, SyntaxError) as e:
        logging.error(f"Cannot identify/process image file for HTML: {image_path}. Error: {type(e).__name__} - {e}")
        return f"Error: Cannot process image: {html_parser.escape(image_path.name)} ({type(e).__name__})"
    except Exception as e:
        logging.error(f"Unexpected error processing image {image_path.name} for HTML: {e}", exc_info=True)
        return f"Error processing image {html_parser.escape(image_path.name)}: {html_parser.escape(str(e))}"

# --- generate_html_report function (logic largely unchanged, uses global SMALL models) ---
def generate_html_report(
    input_image_folder_str: str,
    max_samples_html: int,
    output_html_filename: str = "emotion_visualization_small_models.html"
):
    global device, siglip_model, siglip_processor, emotion_classifiers, neutral_classifier_stats, siglip_device

    if not all([device, siglip_model, siglip_processor, emotion_classifiers]):
        logging.error("Models not loaded. Please run Cell 2 first.")
        return None

    abs_image_folder = Path(input_image_folder_str).resolve()
    if not abs_image_folder.is_dir():
        logging.error(f"Image folder not found: {abs_image_folder}")
        return None

    logging.info(f"\n--- Finding Target Images in {abs_image_folder} (for Small Models HTML) ---")
    image_files_paths = []
    for ext in SUPPORTED_EXTENSIONS:
        image_files_paths.extend(abs_image_folder.rglob(f'*{ext}'))
        image_files_paths.extend(abs_image_folder.rglob(f'*{ext.upper()}'))
    unique_image_files_paths = sorted(list(set(image_files_paths)))
    total_images_found = len(unique_image_files_paths)
    logging.info(f"Found {total_images_found} unique image files.")
    if not unique_image_files_paths: logging.warning("No images found."); return None

    inference_results: List[Dict[str, Any]] = []
    processed_count, error_count = 0, 0
    logging.info("\n--- Running Inference for HTML Report (Small Models) ---")
    classifier_keys = list(emotion_classifiers.keys()) # These are the SMALL model keys
    exclude_keywords_lower = [kw.lower() for kw in EXCLUDE_FROM_SOFTMAX_KEYWORDS]

    with torch.no_grad():
        for img_path_obj in tqdm(unique_image_files_paths, desc="Inferring (Small Models HTML)", unit="image"):
            pil_image = None
            try:
                pil_image = Image.open(img_path_obj).convert("RGB")
                inputs = siglip_processor(images=[pil_image], return_tensors="pt", padding="max_length", truncation=True).to(siglip_device)
                image_features = siglip_model.get_image_features(**inputs)
                embedding_norm = normalized(image_features.cpu().numpy())
                embedding_tensor = torch.from_numpy(embedding_norm).to(device).float()

                if embedding_tensor.shape != (1, EMBEDDING_DIM):
                    logging.warning(f"Embedding shape mismatch for '{img_path_obj.name}'. Skip."); error_count+=1; continue

                predictions: Dict[str, Dict[str, Optional[float]]] = {}
                models_for_softmax, mean_subtracted_scores_list = [], []

                for model_key in classifier_keys:
                    model = emotion_classifiers[model_key]
                    raw_score = float(model(embedding_tensor).item())
                    mean_subtracted_score = raw_score
                    model_neutral_stats = neutral_classifier_stats.get(model_key)
                    if model_neutral_stats and 'mean' in model_neutral_stats:
                        mean_subtracted_score = raw_score - model_neutral_stats['mean']

                    predictions[model_key] = {"raw": raw_score, "mean_subtracted": mean_subtracted_score, "softmax": None}
                    is_excluded = any(keyword in model_key.lower() for keyword in exclude_keywords_lower)
                    if not is_excluded and isinstance(mean_subtracted_score, (int, float)):
                        models_for_softmax.append(model_key)
                        mean_subtracted_scores_list.append(mean_subtracted_score)

                if models_for_softmax:
                    scores_tensor = torch.tensor(mean_subtracted_scores_list, dtype=torch.float32, device=device)
                    softmax_scores_tensor = F.softmax(scores_tensor, dim=0)
                    for i, model_key_sm in enumerate(models_for_softmax): # Renamed to avoid conflict
                        predictions[model_key_sm]["softmax"] = float(softmax_scores_tensor[i].item())

                inference_results.append({"image_path": img_path_obj, "predictions": predictions})
                processed_count += 1
            except Exception as e:
                logging.error(f"Error processing {img_path_obj.name} for HTML: {e}", exc_info=False); error_count += 1
            finally:
                if pil_image: pil_image.close()
                if (processed_count + error_count) % 50 == 0: gc.collect(); torch.cuda.empty_cache() if torch.cuda.is_available() else None

    logging.info(f"--- HTML Inference (Small Models) Complete. Processed: {processed_count}, Errors: {error_count} ---")
    if not inference_results: logging.warning("No results to visualize."); return None

    samples_to_render = inference_results
    if len(inference_results) > max_samples_html:
        samples_to_render = random.sample(inference_results, max_samples_html)
    try: samples_to_render.sort(key=lambda s: s['image_path'].name if s.get('image_path') else "")
    except Exception: pass

    logging.info(f"Generating HTML content for {len(samples_to_render)} samples (Small Models)...")
    # HTML Structure (using the corrected one from previous iteration)
    html_start = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8">
    <title>Emotion Prediction Visualization (Small Models)</title><style>
    body {{ font-family: system-ui, sans-serif; margin: 20px; background-color: #f9fafb; color: #1f2937; }}
    .sample {{ background-color: #fff; border: 1px solid #d1d5db; margin-bottom:20px; padding:15px; border-radius:8px; display:flex; gap:20px; align-items:flex-start;}}
    .image-container img {{ max-width:{IMAGE_DISPLAY_WIDTH}px; height:auto; border:1px solid #e5e7eb; border-radius:6px;}}
    .image-container p {{ font-size:0.8em; color:#6b7280; word-break:break-all; margin-top:5px;}}
    .predictions-container {{ flex-grow:1; min-width:300px;}}
    .dimension-grid {{ display:grid; grid-template-columns:repeat(auto-fill, minmax(300px, 1fr)); gap:6px 10px; margin-top:8px;}}
    .dimension-item {{ background-color:#f9fafb; padding:5px 10px; border-radius:4px; border:1px solid #e5e7eb; font-size:0.85rem; display:flex; justify-content:space-between; align-items:center;}}
    .dimension-name {{ color:#374151; padding-right:8px;}}
    .dimension-score {{ font-weight:500; color:#111827; background-color:#e5e7eb; padding:2px 6px; border-radius:4px; font-family:monospace; min-width:140px; text-align:right;}}
    .mean-sub-score-paren, .raw-score {{ font-size:0.9em; color:#6b7280; margin-left:4px;}}
    .highlight-top1 {{ border-left:4px solid #DC2626; background-color:#FEE2E2 !important;}} .highlight-top1 .dimension-name {{color:#991B1B; font-weight:600;}}
    .highlight-top2 {{ border-left:4px solid #D97706; background-color:#FEF3C7 !important;}} .highlight-top2 .dimension-name {{color:#92400E; font-weight:600;}}
    .highlight-top3 {{ border-left:4px solid #059669; background-color:#D1FAE5 !important;}} .highlight-top3 .dimension-name {{color:#047857; font-weight:500;}}
    .score-na {{color:#6b7280; font-style:italic;}} .error {{color:red;}} h1,h3 {{color:#111827;}} h1{{text-align:center; border-bottom:2px solid #e5e7eb; padding-bottom:10px; margin-bottom:20px;}} h3{{font-size:1.1em; margin-bottom:10px; margin-top:0;}}
    </style></head><body><h1>Emotion Predictions (Small Models)</h1>
    <p>Displaying {len(samples_to_render)} samples. Scores: Softmax (Mean-Sub) (Raw). Highlighting: Top 3 Softmax.</p><hr>"""
    html_end = "</body></html>"; html_body_parts = []

    for i, sample_data in enumerate(tqdm(samples_to_render, desc="Generating HTML parts (Small)")):
        image_path, predictions = sample_data.get('image_path'), sample_data.get('predictions', {})
        img_display_name = image_path.name if image_path else "Unknown"
        img_data_uri_or_error = process_image_for_html(image_path)
        html_body_parts.append(f'<div class="sample"><div class="image-container">')
        if img_data_uri_or_error.startswith('data:image'): html_body_parts.append(f'<img src="{img_data_uri_or_error}" alt="{html_parser.escape(img_display_name)}">')
        else: html_body_parts.append(f'<div class="error">{img_data_uri_or_error}</div>')
        display_path_str = str(image_path.relative_to(Path("/content"))) if image_path and image_path.is_relative_to(Path("/content")) else str(image_path)
        html_body_parts.append(f'<p>./{html_parser.escape(display_path_str)}</p></div>')
        html_body_parts.append(f'<div class="predictions-container"><h3>{i+1}. {html_parser.escape(img_display_name)}</h3><div class="dimension-grid">')

        valid_softmax_scores = []
        for dk, sd in predictions.items(): # dim_key, score_data
            s_val = sd.get('softmax')
            if isinstance(s_val, (int, float)): valid_softmax_scores.append((dk, s_val))
        valid_softmax_scores.sort(key=lambda item: item[1], reverse=True)
        top_ranks = {mk: r for r, (mk, _) in enumerate(valid_softmax_scores[:3], 1)} # model_key, rank

        for dim_key_html in sorted(predictions.keys()): # Renamed
            score_data_html = predictions.get(dim_key_html, {}) # Renamed
            raw_s, mean_sub_s, softmax_s = score_data_html.get('raw'), score_data_html.get('mean_subtracted'), score_data_html.get('softmax')
            is_excluded = any(kw in dim_key_html.lower() for kw in exclude_keywords_lower)
            score_str_parts, na_class = [], ""
            if not is_excluded and isinstance(softmax_s, (int, float)):
                score_str_parts.append(f"{softmax_s:.3f}")
                mean_sub_text = f"{mean_sub_s:.2f}" if isinstance(mean_sub_s, (int,float)) else "N/A"
                score_str_parts.append(f'<span class="mean-sub-score-paren">({mean_sub_text})</span>')
                raw_text = f"{raw_s:.1f}" if isinstance(raw_s, (int,float)) else "N/A"
                score_str_parts.append(f'<span class="raw-score">({raw_text})</span>')
            else:
                if isinstance(mean_sub_s, (int, float)): score_str_parts.append(f"{mean_sub_s:.2f}")
                else: score_str_parts.append("N/A"); na_class = "score-na"
                raw_text = f"{raw_s:.1f}" if isinstance(raw_s, (int,float)) else "N/A"
                score_str_parts.append(f'<span class="raw-score">({raw_text})</span>')

            final_score_str = " ".join(score_str_parts)
            rank = top_ranks.get(dim_key_html); hl_class = ""
            if rank == 1: hl_class = "highlight-top1"
            elif rank == 2: hl_class = "highlight-top2"
            elif rank == 3: hl_class = "highlight-top3"
            html_body_parts.append(f'<div class="dimension-item {hl_class}"><span class="dimension-name">{html_parser.escape(dim_key_html)}</span> <span class="dimension-score {na_class}">{final_score_str}</span></div>')
        html_body_parts.append('</div></div></div>')

    full_html = html_start + "\n".join(html_body_parts) + html_end
    return full_html

# --- Run HTML Generation ---
if not Path(IMAGE_FOLDER_FOR_HTML).is_dir():
    print(f"ERROR: Image folder '{IMAGE_FOLDER_FOR_HTML}' not found.")
else:
    print(f"Generating HTML report for Small Models in: {IMAGE_FOLDER_FOR_HTML}")
    html_content = generate_html_report(IMAGE_FOLDER_FOR_HTML, MAX_SAMPLES_FOR_HTML)
    if html_content:
        clear_output(wait=True)
        display(HTML(html_content))
        print(f"HTML report (Small Models) displayed for '{IMAGE_FOLDER_FOR_HTML}'.")
    else:
        print(f"Failed to generate HTML report for Small Models from '{IMAGE_FOLDER_FOR_HTML}'.")

In [ ]:
#@title ## Cell 4: Generate JSON Annotations for Image Folder (using Empathic-Insight-Face-Small)
#@markdown Uses the SMALL models loaded in Cell 2.
#@markdown 1. Ensure images are in a folder (e.g., default demo images from Cell 1).
#@markdown 2. Enter the **full path** to this image folder.
#@markdown 3. Click Run. JSON files saved in subfolder.

IMAGE_FOLDER_FOR_JSON = str(DEMO_IMAGES_DIR) #@param {type:"string"}
OUTPUT_JSON_SUBFOLDER_NAME = "json_annotations_small_models"

# --- generate_json_annotations function (logic largely unchanged, uses global SMALL models) ---
def generate_json_annotations(
    input_image_folder_str: str,
    output_json_subfolder_name_str: str # Renamed
):
    global device, siglip_model, siglip_processor, emotion_classifiers, neutral_classifier_stats, MODEL_KEY_TO_JSON_KEY_MAP, siglip_device, HF_REPO_ID_SMALL

    if not all([device, siglip_model, siglip_processor, emotion_classifiers, MODEL_KEY_TO_JSON_KEY_MAP]):
        logging.error("Models/mappings not loaded. Run Cell 2."); return

    abs_image_folder = Path(input_image_folder_str).resolve()
    if not abs_image_folder.is_dir(): logging.error(f"Image folder not found: {abs_image_folder}"); return

    abs_output_json_folder = abs_image_folder / output_json_subfolder_name_str
    abs_output_json_folder.mkdir(parents=True, exist_ok=True)
    logging.info(f"JSON annotations (Small Models) will be saved to: {abs_output_json_folder}")

    logging.info(f"\n--- Finding Target Images in {abs_image_folder} (for Small Models JSON) ---")
    image_files_paths = []
    for ext in SUPPORTED_EXTENSIONS:
        image_files_paths.extend(abs_image_folder.rglob(f'*{ext}'))
        image_files_paths.extend(abs_image_folder.rglob(f'*{ext.upper()}'))
    unique_image_files_paths = sorted(list(set(p for p in image_files_paths if p.is_file() and p.parent == abs_image_folder)))
    total_images_found = len(unique_image_files_paths)
    logging.info(f"Found {total_images_found} top-level images for JSON annotation (Small Models).")
    if not unique_image_files_paths: logging.warning("No top-level images found."); return

    processed_count, error_count, json_saved_count = 0, 0, 0
    logging.info("\n--- Running Inference and Saving JSON (Small Models) ---")
    internal_classifier_keys = list(emotion_classifiers.keys())

    with torch.no_grad():
        for img_path_obj_json in tqdm(unique_image_files_paths, desc="Inferring/Saving JSON (Small)", unit="image"): # Renamed
            pil_image = None
            try:
                pil_image = Image.open(img_path_obj_json).convert("RGB")
                inputs = siglip_processor(images=[pil_image], return_tensors="pt", padding="max_length", truncation=True).to(siglip_device)
                image_features = siglip_model.get_image_features(**inputs)
                embedding_norm = normalized(image_features.cpu().numpy())
                embedding_tensor = torch.from_numpy(embedding_norm).to(device).float()

                if embedding_tensor.shape != (1, EMBEDDING_DIM):
                    logging.warning(f"Embedding shape mismatch for '{img_path_obj_json.name}'. Skip."); error_count+=1; continue

                predictions_output_dict: Dict[str, float] = {}
                for internal_model_key_json in internal_classifier_keys: # Renamed
                    model_instance = emotion_classifiers[internal_model_key_json]
                    raw_score = float(model_instance(embedding_tensor).item())
                    final_score = raw_score
                    model_neutral_s_dict = neutral_classifier_stats.get(internal_model_key_json) # Renamed
                    if model_neutral_s_dict and 'mean' in model_neutral_s_dict:
                        final_score = raw_score - model_neutral_s_dict['mean']

                    json_output_key = MODEL_KEY_TO_JSON_KEY_MAP.get(internal_model_key_json)
                    if json_output_key: predictions_output_dict[json_output_key] = final_score
                    else: logging.warning(f"No JSON mapping for '{internal_model_key_json}'. Score excluded for {img_path_obj_json.name}.")

                json_key_list = [abs_image_folder.name, HF_REPO_ID_SMALL.split('/')[-1], img_path_obj_json.name] # Use SMALL repo ID
                output_data = {"key": json_key_list, "value": predictions_output_dict}
                json_filename = img_path_obj_json.stem + ".json"
                output_json_path = abs_output_json_folder / json_filename

                try:
                    with open(output_json_path, 'w', encoding='utf-8') as f_json: json.dump(output_data, f_json, indent=2)
                    json_saved_count +=1
                except Exception as e_json: logging.error(f"Error saving JSON for {img_path_obj_json.name}: {e_json}"); error_count +=1
                processed_count += 1
            except Exception as e:
                logging.error(f"Error processing {img_path_obj_json.name} for JSON: {type(e).__name__} - {e}", exc_info=False); error_count += 1
            finally:
                if pil_image: pil_image.close()
                if processed_count > 0 and processed_count % 100 == 0: gc.collect(); torch.cuda.empty_cache() if torch.cuda.is_available() else None

    logging.info(f"--- JSON Inference/Save (Small Models) Complete ---")
    logging.info(f"Processed: {processed_count}/{total_images_found} images.")
    logging.info(f"Saved: {json_saved_count} JSON files to '{abs_output_json_folder}'.")
    if error_count > 0: logging.warning(f"Errors for: {error_count} images/ops during JSON generation (Small Models).")

# --- Run JSON Generation ---
if not Path(IMAGE_FOLDER_FOR_JSON).is_dir():
    print(f"ERROR: Image folder '{IMAGE_FOLDER_FOR_JSON}' not found.")
else:
    print(f"Generating JSON annotations for Small Models in: {IMAGE_FOLDER_FOR_JSON}")
    print(f"JSON files will be saved into subfolder '{OUTPUT_JSON_SUBFOLDER_NAME}'.")
    generate_json_annotations(IMAGE_FOLDER_FOR_JSON, OUTPUT_JSON_SUBFOLDER_NAME)
    print(f"\nJSON annotation (Small Models) finished for '{IMAGE_FOLDER_FOR_JSON}'.")
    print(f"Check '{Path(IMAGE_FOLDER_FOR_JSON) / OUTPUT_JSON_SUBFOLDER_NAME}' for outputs.")
    print(f"Example list: !ls -l {Path(IMAGE_FOLDER_FOR_JSON) / OUTPUT_JSON_SUBFOLDER_NAME}/*.json")